# VGG
计算机视觉是一直深度学习的主战场，从这里我们将接触到近几年非常流行的卷积网络结构，网络结构由浅变深，参数越来越多，网络有着更多的跨层链接，首先我们先介绍一个数据集 cifar10，我们将以此数据集为例介绍各种卷积网络的结构。

## CIFAR 10
cifar 10 这个数据集一共有 50000 张训练集，10000 张测试集，两个数据集里面的图片都是 png 彩色图片，图片大小是 32 x 32 x 3，一共是 10 分类问题，分别为飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船和卡车。这个数据集是对网络性能测试一个非常重要的指标，可以说如果一个网络在这个数据集上超过另外一个网络，那么这个网络性能上一定要比另外一个网络好，目前这个数据集最好的结果是 95% 左右的测试集准确率。

![image.png](attachment:9bc17506-cd65-45d3-9307-88ed3629b8bb.png)

你能用肉眼对这些图片进行分类吗？

cifar 10 已经被 pytorch 内置了，使用非常方便，只需要调用 `torchvision.datasets.CIFAR10` 就可以了

## VGGNet
vggNet 是第一个真正意义上的深层网络结构，其是 ImageNet2014年的冠军，得益于 python 的函数和循环，我们能够非常方便地构建重复结构的深层网络。https://blog.csdn.net/One2332x/article/details/122171029

vgg 的网络结构非常简单，就是不断地堆叠卷积层和池化层，下面是一个简单的图示

![](https://ws4.sinaimg.cn/large/006tNc79ly1fmpk5smtidj307n0dx3yv.jpg)

vgg 几乎全部使用 3 x 3 的卷积核以及 2 x 2 的池化层，使用小的卷积核进行多层的堆叠和一个大的卷积核的感受野是相同的，同时小的卷积核还能减少参数，同时可以有更深的结构。

vgg 的一个关键就是使用很多层 3 x 3 的卷积然后再使用一个最大池化层，这个模块被使用了很多次，下面我们照着这个结构来写一写

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

from torchsummary import summary
from torchvision import transforms as tfs

我们可以定义一个 vgg 的 block，传入三个参数，第一个是模型层数，第二个是输入的通道数，第三个是输出的通道数，第一层卷积接受的输入通道就是图片输入的通道数，然后输出最后的输出通道数，后面的卷积接受的通道数就是最后的输出通道数

In [ ]:
#(卷积层数,输入的通道数,输出的通道数)
def vgg_block(num_convs, in_channels, out_channels):
    net = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), nn.ReLU(True)] # 定义第一层
    
    for i in range(num_convs-1): # 定义后面的很多层
        net.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        net.append(nn.ReLU(True))
        
    net.append(nn.MaxPool2d(2, 2)) # 定义池化层
    return nn.Sequential(*net)

我们可以将模型打印出来看看结构

In [ ]:
block_demo = vgg_block(3, 64, 128)
print(block_demo)

Sequential(
  (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [ ]:
# 首先定义输入为 (1, 64, 300, 300)
input_demo = Variable(torch.zeros(1, 64, 300, 300))
output_demo = block_demo(input_demo)
print(output_demo.shape)

torch.Size([1, 128, 150, 150])


/root/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


可以看到输出就变为了 (1, 128, 150, 150)，可以看到经过了这一个 vgg block，输入大小被减半，通道数变成了 128

下面我们定义一个函数对这个 vgg block 进行堆叠

In [ ]:
def vgg_stack(num_convs, channels):
    net = []
    for n, c in zip(num_convs, channels):
        in_c = c[0]
        out_c = c[1]
        net.append(vgg_block(n, in_c, out_c))
    return nn.Sequential(*net)

作为实例，我们定义一个稍微简单一点的 vgg 结构，其中有 8 个卷积层

In [ ]:
#共有5个vgg_block，每个vgg_block的卷积层数依次为(1, 1, 2, 2, 2)
#每个vgg_block对应的（输入的通道数,输出的通道数）依次为
#((3, 64), (64, 128), (128, 256), (256, 512), (512, 512))
vgg_net = vgg_stack((1, 1, 2, 2, 3), ((3, 64), (64, 128), (128, 256), (256, 512), (512, 512)))
print(vgg_net)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(ker

我们可以看到网络结构中有个 5 个 最大池化，说明图片的大小会减少 5 倍，我们可以验证一下，输入一张 256 x 256 的图片看看结果是什么

In [ ]:
test_x = Variable(torch.zeros(1, 3, 256, 256))
test_y = vgg_net(test_x)
print(test_y.shape)

torch.Size([1, 512, 8, 8])


可以看到图片减小了 $2^5$ 倍，最后再加上几层全连接，就能够得到我们想要的分类输出

In [ ]:
#使用批量标准化
class vgg(nn.Module):
    def __init__(self):
        super(vgg, self).__init__()
        self.feature = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(64),#(输出通道数)
                                     nn.ReLU(True),
                                     nn.MaxPool2d(2, 2),
                                     
                                     nn.Conv2d(64, 128, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(128),#(输出通道数)
                                     nn.ReLU(True),
                                     nn.MaxPool2d(2, 2),
                                     
                                     nn.Conv2d(128, 256, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(256),#(输出通道数)
                                     nn.ReLU(True),
                            
                                     nn.Conv2d(256, 256, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(256),#(输出通道数)
                                     nn.ReLU(True),
                                     nn.MaxPool2d(2, 2),
                                     
                                     nn.Conv2d(256, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),#(输出通道数)
                                     nn.ReLU(True),
                                     
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),#(输出通道数)
                                     nn.ReLU(True),
                                     nn.MaxPool2d(2, 2),
                                     
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),#(输出通道数)
                                     nn.ReLU(True),
                                    
                                     
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),#(输出通道数)
                                     nn.ReLU(True),
                                     
                                     
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),#(输出通道数)
                                     nn.ReLU(True),
                                     nn.MaxPool2d(2, 2),
                                     
                                     
                                     
                                     
                                     
                                     
                                    )
        self.fc = nn.Sequential(
            nn.Dropout(0.2),# Dropout 放在需要drop神经元的层之前，传入每个神经元被舍弃的概率
            nn.Linear(512, 100),
            nn.ReLU(True),
            nn.Linear(100, 10)
        )
    def forward(self, x):
        #print(x.shape)#torch.Size([64, 3, 32, 32])
        x = self.feature(x)
        #print(x.shape)#torch.Size([64, 512, 1, 1])
        x = x.view(x.shape[0], -1)
        #print(x.shape)#torch.Size([64, 512])
        x = self.fc(x)
        return x

In [ ]:
net0 = vgg()

In [ ]:
if torch.cuda.is_available():
      net0 = net0.cuda()

summary(net0, (3, 32, 32)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 8, 8]         295,168
      BatchNorm2d-10            [-1, 256, 8, 8]             512
             ReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 256, 8, 8]         590,080
      BatchNorm2d-13            [-1, 256, 8, 8]             512
             ReLU-14            [-1, 25

然后我们可以训练我们的模型看看在 cifar10 上的效果

In [ ]:
from utils import train


#使用数据增强
def train_tf(x):
    im_aug = tfs.Compose([
        #tfs.Resize(32),
        tfs.RandomHorizontalFlip(),
        tfs.RandomRotation(45),
        tfs.RandomVerticalFlip(),
        #tfs.RandomCrop(20),
        tfs.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
        tfs.ToTensor(),
        #Normalize对每个通道执行以下操作：image =（图像-平均值）/ std在您的情况下，参数mean，std分别以0.5和0.5的形式传递。
        #这将使图像在[-1,1]范围内归一化。
        tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

def test_tf(x):
    im_aug = tfs.Compose([
        tfs.ToTensor(),
        #Normalize对每个通道执行以下操作：image =（图像-平均值）/ std在您的情况下，参数mean，std分别以0.5和0.5的形式传递。
        #这将使图像在[-1,1]范围内归一化。
        tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        
    ])
    x = im_aug(x)
    return x


train_set = CIFAR10('./data', train=True, transform=train_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=test_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128,shuffle=False)

net = vgg()
#optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
#optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2, momentum=0.9,weight_decay=1e-4) # 添加权重衰减系数
#if torch.cuda.is_available():
      #net = net.cuda()
#optimizer = torch.optim.Adagrad(net.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

In [ ]:

#tensorboard设置，将路径设置为../outpue
from torch.utils.tensorboard import SummaryWriter
writer0 = SummaryWriter(log_dir = '../output')#将条目直接写入output文件夹中的事件文件以供 TensorBoard 使用

import torchvision

In [ ]:
#学习率修改函数
def set_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:

def train(net, train_data, valid_data, num_epochs, optimizer, criterion):
       
    if torch.cuda.is_available():
        net = net.cuda()
    prev_time = datetime.now()
    for epoch in range(num_epochs):
        if epoch == 40:
            set_learning_rate(optimizer, 0.005) # 第40 个epoch修改学习率为 0.005
        
        train_loss = 0
        train_acc = 0
        net = net.train()
        for im, label in train_data:
            if torch.cuda.is_available():
                im = Variable(im.cuda())  # (bs, 3, h, w)
                label = Variable(label.cuda())  # (bs, h, w)
            else:
                im = Variable(im)
                label = Variable(label)
            # forward
            output = net(im)
            loss = criterion(output, label)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()
            train_acc += get_acc(output, label)
                     
        cur_time = datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = 0
            valid_acc = 0
            net = net.eval()
            for im, label in valid_data:
                if torch.cuda.is_available():
                    im = Variable(im.cuda(), volatile=True)
                    label = Variable(label.cuda(), volatile=True)
                else:
                    im = Variable(im, volatile=True)
                    label = Variable(label, volatile=True)
                output = net(im)
                loss = criterion(output, label)
                valid_loss += loss.data.item()
                valid_acc += get_acc(output, label)
            epoch_str = (
                "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), valid_loss / len(valid_data),
                   valid_acc / len(valid_data)))
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %
                         (epoch, train_loss / len(train_data),
                          train_acc / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str)
        #增加将数据写入Writer
        
        writer0.add_scalar('Train Loss',  train_loss / len(train_data),  epoch)#添加标量数据Train Loss到事件文件writer0中
        writer0.add_scalar('Train Acc',   train_acc / len(train_data),  epoch)
        writer0.add_scalar('Valid Loss',  valid_loss / len(valid_data),  epoch)
        writer0.add_scalar('Valid Acc',   valid_acc / len(valid_data),  epoch)
    writer0.close()

In [ ]:

#from utils import train
from datetime import datetime

def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().data.item()
    return num_correct / total

train(net, train_data, test_data, 50, optimizer, criterion)

/tmp/ipykernel_1130/3668878487.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
/tmp/ipykernel_1130/3668878487.py:42: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 2.009950, Train Acc: 0.250020, Valid Loss: 1.962028, Valid Acc: 0.263054, Time 00:01:07
Epoch 1. Train Loss: 1.703365, Train Acc: 0.375360, Valid Loss: 1.534034, Valid Acc: 0.433643, Time 00:01:10


可以看到，跑完 20 次，vgg 能在 cifar 10 上取得 76% 左右的测试准确率